In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ast
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import helpers
import ollama

#### Get ESCO embeddings

In [ ]:
esco = pd.read_csv('datasets/skills_en.csv')

In [ ]:
esco_skills = esco['preferredLabel'].tolist()

In [ ]:
response = ollama.embed(model="nomic-embed-text", input=esco_skills)

In [ ]:
esco['embeddings'] = response['embeddings']

In [ ]:
esco.to_parquet('processed/esco_skills.parquet', index=False)

In [3]:
esco = pd.read_parquet('processed/esco_skills.parquet')

#### Preprocess dice

In [ ]:
dice = pd.read_csv('datasets/dice_com-job_us_sample.csv')

In [ ]:
dice

In [ ]:
dice['skills'] = dice['skills'].fillna('')

In [ ]:
dice['skills'] = dice['skills'].str.lower().str.strip()

In [ ]:
dice[dice['skills'].str.contains('see')]

In [ ]:
dice[~dice['skills'].str.contains('see')]['skills']

In [ ]:
dice_w_skills = helpers.get_list(data=dice, 
                                 col='jobdescription', 
                                 doc_type='job', 
                                 max_workers=10,
                                 model="mistral:instruct")

In [ ]:
dice_w_skills

In [ ]:
dice_w_skills.to_parquet('processed/dice_job_descriptions_with_skills.parquet', index=False)

In [ ]:
dice_w_skills = pd.read_parquet('processed/dice_job_descriptions_with_skills.parquet')

In [ ]:
dice_w_skills = helpers.fill_missing_skills(data=dice_w_skills,
                                           skills_col='skills',
                                           doc_type='job',
                                           max_workers=3)

In [ ]:
dice_w_skills

In [ ]:
matched = helpers.match_all_skills_con(dice_w_skills, esco, threshold=0.8)

In [ ]:
matched.to_parquet('processed/dice_job_descriptions_matched.parquet', index=False)

In [ ]:
jobs_matched = pd.read_parquet('processed/dice_job_descriptions_matched.parquet')

In [ ]:
jobs_matched['matched_skills_ordered'] = jobs_matched['matched_skills'].apply(lambda x: sorted(set(x)))

In [ ]:
skill_embeddings = helpers.embed_skills_list(jobs_matched['matched_skills'].tolist())

In [ ]:
skill_embeddings_ordered = helpers.embed_skills_list(jobs_matched['matched_skills_ordered'].tolist())

In [ ]:
jobs_matched['skill_embeddings'] = skill_embeddings['embeddings']

In [ ]:
jobs_matched['skill_embeddings_ordered'] = skill_embeddings_ordered['embeddings']

In [ ]:
matched

In [ ]:
jobs_matched.to_parquet('processed/dice_job_descriptions_embeddings.parquet', index=False)

In [5]:
matched = pd.read_parquet('processed/dice_job_descriptions_embeddings.parquet')

In [6]:
matched

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id,extracted_skills,matched_skills,matched_skills_ordered,skill_embeddings,skill_embeddings_ordered
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,None,see below,418ff92580b270ef4e7c14f0ddfc36b4,"[Selenium, Java, Data Structures, Object Orien...","[Java (computer programming), information stru...","[Groovy, Java (computer programming), Oracle R...","[0.00816865, 0.04228201, -0.12763296, -0.06408...","[0.0012203407, 0.048476767, -0.1365127, -0.069..."
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,None,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9,"[Incident Response, Information Security Asses...","[investigate security issues, documentation ty...","[documentation types, investigate security iss...","[0.018865215, 0.036568727, -0.16978557, 0.0041...","[0.018206175, 0.013558769, -0.1604094, 0.00365..."
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,None,"enterprise solutions architecture, business in...",46baa1f69ac07779274bcd90b85d9a72,"[Business Intelligence, Data Analysis, Data Wa...","[business intelligence, perform data analysis,...","[apply change management, business intelligenc...","[0.011157077, -0.003909247, -0.17394324, 0.010...","[0.011065557, -0.012342164, -0.18372022, 0.000..."
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,None,please see job description,3941b2f206ae0f900c4fba4ac0b18719,"[Java, JDBC, Multithreading, Linux/AIX/Unix, S...","[Java (computer programming), SQL, information...","[Java (computer programming), SQL, communicati...","[0.000571077, 0.026021862, -0.13816436, -0.033...","[0.00030429946, 0.02412445, -0.13305221, -0.03..."
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,None,"configuration management, developer, linux, ma...",45efa1f6bc65acc32bbbb953a1ed13b7,"[DevOps, Project Management, Scripting, Config...","[DevOps, project management, project configura...","[Ansible, DevOps, manipulate puppets, project ...","[-0.017652616, 0.057342496, -0.13930643, -0.05...","[-0.010731407, 0.060096607, -0.1377153, -0.060..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,None,"ui/ux mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2,"[Python, Project Management, Data Analysis, UI...","[Python (computer programming), project manage...","[Agile development, Python (computer programmi...","[-0.036120027, 0.06199489, -0.13738641, -0.028...","[-0.011015506, 0.052001737, -0.14096637, 0.005..."
21996,https://www.dice.com/jobs/detail/Senior-Front-...,Omega Solutions

In [17]:
matched_avg = helpers.average_skills(matched, esco)

100%|██████████| 22000/22000 [00:01<00:00, 16887.10it/s]


In [18]:
matched_avg

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id,extracted_skills,matched_skills,matched_skills_ordered,skill_embeddings,skill_embeddings_ordered,avg_skill_embedding
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,None,see below,418ff92580b270ef4e7c14f0ddfc36b4,"[Selenium, Java, Data Structures, Object Orien...","[Java (computer programming), information stru...","[Groovy, Java (computer programming), Oracle R...","[0.00816865, 0.04228201, -0.12763296, -0.06408...","[0.0012203407, 0.048476767, -0.1365127, -0.069...","[0.012229671666666667, 0.02882406688888889, -0..."
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,None,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9,"[Incident Response, Information Security Asses...","[investigate security issues, documentation ty...","[documentation types, investigate security iss...","[0.018865215, 0.036568727, -0.16978557, 0.0041...","[0.018206175, 0.013558769, -0.1604094, 0.00365...","[0.02598377266666667, 0.018216321833333337, -0..."
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,None,"enterprise solutions architecture, business in...",46baa1f69ac07779274bcd90b85d9a72,"[Business Intelligence, Data Analysis, Data Wa...","[business intelligence, perform data analysis,...","[apply change management, business intelligenc...","[0.011157077, -0.003909247, -0.17394324, 0.010...","[0.011065557, -0.012342164, -0.18372022, 0.000...","[0.007513185861666667, -0.007068083333333332, ..."
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,None,please see job description,3941b2f206ae0f900c4fba4ac0b18719,"[Java, JDBC, Multithreading, Linux/AIX/Unix, S...","[Java (computer programming), SQL, information...","[Java (computer programming), SQL, communicati...","[0.000571077, 0.026021862, -0.13816436, -0.033...","[0.00030429946, 0.02412445, -0.13305221, -0.03...","[0.010603966424999998, 0.014678564500000001, -..."
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,None,"configuration management, developer, linux, ma...",45efa1f6bc65acc32bbbb953a1ed13b7,"[DevOps, Project Management, Scripting, Config...","[DevOps, project management, project configura...","[Ansible, DevOps, manipulate puppets, project ...","[-0.017652616, 0.057342496, -0.13930643, -0.05...","[-0.010731407, 0.060096607, -0.1377153, -0.060...","[0.0009501145119999992, 0.033562203, -0.132539..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,None,"ui/ux mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2,"[Python, Project Management, Data Analysis, UI

In [19]:
matched_avg.to_parquet('processed/dice_job_descriptions_embeddings.parquet', index=False)

#### Preprocessing Resume

In [ ]:
resume = pd.read_csv('datasets/Resume.csv')

In [ ]:
resume_w_skills = helpers.get_list(data=resume, 
                                   col='Resume_str',
                                   doc_type='resume', 
                                   max_workers=10,
                                   model="mistral:instruct")

In [ ]:
resume_w_skills.to_parquet('processed/resume_w_skills.parquet', index=False)

In [ ]:
resume = pd.read_parquet('processed/resume_w_skills.parquet')

In [ ]:
resume = helpers.fill_missing_skills(resume, esco)

In [ ]:
resume

In [ ]:
resume['length'] = resume['Resume_str'].str.len()

In [ ]:
resume[['Resume_str', 'extracted_skills', 'length']]

In [ ]:
print(resume['Resume_str'].iloc[94])

In [ ]:
skills = resume['extracted_skills'].iloc[0]

In [ ]:
matched = helpers.match_closest_skills(skills, esco, threhold=0.8)

In [ ]:
matched_list, matched_scores = matched

In [ ]:
matched_list

In [ ]:
matched_scores

In [ ]:
comp_df = pd.DataFrame(columns=['skill', 'matched_skill', 'score'], data=list(zip(skills, matched_list, matched_scores)))

In [ ]:
comp_df

In [ ]:
resume_matched = helpers.match_all_skills_con(resume, esco, threshold=0.8)

In [ ]:
resume_matched

In [ ]:
resume_matched.to_parquet('processed/resume_matched.parquet', index=False)

In [ ]:
resume_matched = pd.read_parquet('processed/resume_matched.parquet')

In [ ]:
resume_matched['matched_skills_ordered'] = resume_matched['matched_skills'].apply(lambda x: sorted(set(x)))

In [ ]:
skill_embeddings = helpers.embed_skills_list(resume_matched['matched_skills'].tolist())

In [ ]:
skill_embeddings_ordered = helpers.embed_skills_list(resume_matched['matched_skills_ordered'].tolist())

In [ ]:
resume_matched['skill_embeddings'] = skill_embeddings['embeddings']

In [ ]:
resume_matched['skill_embeddings_ordered'] = skill_embeddings_ordered['embeddings']

In [ ]:
resume_matched

In [ ]:
resume_matched.to_parquet('processed/resume_embeddings.parquet', index=False)

In [20]:
resume_matched = pd.read_parquet('processed/resume_embeddings.parquet')

In [22]:
resume_matched_avg =helpers.average_skills(resume_matched, esco)

100%|██████████| 2484/2484 [00:00<00:00, 12114.34it/s]


In [23]:
resume_matched_avg.to_parquet('processed/resume_embeddings.parquet', index=False)